# 01 Data Ingestion and Quality Checks

## Objectives

- Load the raw dataset from the versioned folder
- Perform basic data quality checks and standardization
- Export a cleaned dataset to the processed folder

## Inputs

- data/raw/v1/environmental_trends.csv

## Outputs

- data/processed/v1/environmental_trends_clean.csv

## Additional Comments

- Keep all changes reproducible and logged in the notebook

## Notebook layout

- Section 1: Setup and load raw data
- Section 2: Data quality checks
- Section 3: Save cleaned output

## Business context

This notebook supports all downstream analysis and the Streamlit dashboard by ensuring the data foundation is reliable and reproducible.

## Purpose and Context

This notebook is the first step in our Global Environmental Trends analysis pipeline. It ensures our data foundation is solid before we draw any conclusions or make predictions.

The connection to project guidelines centers on three key areas. For ethics, transparent data quality checks prevent misleading conclusions that could affect public understanding. For communication, clear documentation helps both technical reviewers and non-technical stakeholders understand our process. For project planning, establishing a clean, versioned dataset enables reproducible analysis and future updates.

What makes this approach responsible is that we document all quality issues rather than just the ones we fix. We preserve the raw data unchanged by separating raw from processed files. We version outputs so anyone can reproduce our results. We make the cleaning process visible with no hidden transformations.

---

## Section 1 - Setup and load raw data

This section sets the project root as the working directory and loads the raw CSV for inspection.

# Change working directory

Notebooks are stored in the `jupyter_notebooks/` subfolder. This cell checks the current working directory and navigates to the project root (`global_env_trend/`) if needed, ensuring relative paths to `data/` work correctly.

In [1]:
import os
from pathlib import Path

current_dir = os.getcwd()
current_dir

'c:\\Users\\sergi\\OneDrive\\Documents\\Code Institute Data analytics\\Capstone project 3\\Global_environmental_trends_2000_2024\\global_env_trend\\jupyter_notebooks'

In [2]:
# Navigate to project root - portable approach that works on any clone
current = Path.cwd()

if not (current / "data" / "processed" / "v1").exists():
    if (current.parent / "data" / "processed" / "v1").exists():
        os.chdir(current.parent)

print(f"Working directory: {os.getcwd()}")

Working directory: c:\Users\sergi\OneDrive\Documents\Code Institute Data analytics\Capstone project 3\Global_environmental_trends_2000_2024\global_env_trend


In [3]:
print(f"Current directory: {os.getcwd()}")
print(f"Data folder exists: {os.path.exists('data')}")

Current directory: c:\Users\sergi\OneDrive\Documents\Code Institute Data analytics\Capstone project 3\Global_environmental_trends_2000_2024\global_env_trend
Data folder exists: True


# Load raw data

In [4]:
import pandas as pd
import numpy as np

raw_path = "data/raw/v1/environmental_trends.csv"
try:
    df = pd.read_csv(raw_path)
    print(f"Loaded {len(df)} rows and {len(df.columns)} columns")
except FileNotFoundError:
    raise FileNotFoundError(f"Raw data not found at {raw_path}. Ensure the file exists.")

df.head()

Loaded 156 rows and 10 columns


,Year,Country,Avg_Temperature_degC,CO2_Emissions_tons_per_capita,Sea_Level_Rise_mm,Rainfall_mm,Population,Renewable_Energy_pct,Extreme_Weather_Events,Forest_Area_pct
0,2000,United States,13.5,20.2,0,715,282500000,6.2,38,33.1
1,2000,China,12.8,2.7,0,645,1267000000,16.5,24,18.8
2,2000,Germany,9.3,10.1,0,700,82200000,6.6,12,31.8
3,2000,Brazil,24.9,1.9,0,1760,175000000,83.7,18,65.4
4,2000,Australia,21.7,17.2,0,534,19200000,8.8,11,16.2


**What this code does:**

Loads the raw CSV from the versioned folder and previews the first rows to verify columns and types before any cleaning.

## Section 2 - Data quality checks

We assess missing values, duplicates, and data types to validate reliability before analysis.

# Quality checks

**What we're checking and why:**

Data quality is the foundation of reliable analysis. Before we can trust any insights or predictions, we need to verify that our data is complete (checking for missing values that could bias results), accurate (reviewing data types to ensure calculations will work correctly), and consistent (identifying duplicates that could skew statistics).

From an ethical perspective, poor data quality can lead to misleading conclusions that affect public understanding of climate issues. By documenting these checks, we ensure transparency and accountability.

In [5]:
# Data types and non-null counts
print("=== Data Types and Shape ===")
print(f"Shape: {df.shape}")
print(f"\nExpected columns: Year, Country, Avg_Temperature_degC, CO2_Emissions_tons_per_capita,")
print(f"  Sea_Level_Rise_mm, Rainfall_mm, Population, Renewable_Energy_pct,")
print(f"  Extreme_Weather_Events, Forest_Area_pct")
print(f"\nActual columns: {list(df.columns)}")
print()
df.info()

=== Data Types and Shape ===
Shape: (156, 10)

Expected columns: Year, Country, Avg_Temperature_degC, CO2_Emissions_tons_per_capita,
  Sea_Level_Rise_mm, Rainfall_mm, Population, Renewable_Energy_pct,
  Extreme_Weather_Events, Forest_Area_pct

Actual columns: ['Year', 'Country', 'Avg_Temperature_degC', 'CO2_Emissions_tons_per_capita', 'Sea_Level_Rise_mm', 'Rainfall_mm', 'Population', 'Renewable_Energy_pct', 'Extreme_Weather_Events', 'Forest_Area_pct']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Year                           156 non-null    int64  
 1   Country                        156 non-null    object 
 2   Avg_Temperature_degC           156 non-null    float64
 3   CO2_Emissions_tons_per_capita  156 non-null    float64
 4   Sea_Level_Rise_mm              156 non-null    int64  
 5   Rainfall_mm  

**What this shows:**

The info method reveals column names and data types (are temperatures stored as numbers?), non-null counts (how many valid values per column?), and memory usage (dataset size). Look for unexpected types such as numbers stored as text or excessive missing data.

In [6]:
# Missing values analysis
print("=== Missing Values ===")
missing = df.isna().sum().sort_values(ascending=False)
missing_pct = (missing / len(df) * 100).round(2)
missing_report = pd.DataFrame({"Missing Count": missing, "Percentage (%)": missing_pct})
print(missing_report[missing_report["Missing Count"] > 0].to_string() if missing.sum() > 0 else "No missing values found.")
print(f"\nTotal missing cells: {missing.sum()} out of {df.shape[0] * df.shape[1]} ({missing.sum() / (df.shape[0] * df.shape[1]) * 100:.2f}%)")

=== Missing Values ===
No missing values found.

Total missing cells: 0 out of 1560 (0.00%)


**Interpreting missing values:**

Low counts below 5% are usually acceptable and may indicate incomplete reporting for specific countries or years. High counts above 20% require caution, so consider whether the variable can still be used reliably. Also watch for patterns where missing data is concentrated in certain countries or years, as this may introduce bias.

The action we take is to document any columns with significant missing data and note this as a limitation in the final dashboard.

In [7]:
# Duplicate analysis
print("=== Duplicate Analysis ===")
n_dupes = df.duplicated().sum()
print(f"Fully duplicate rows: {n_dupes}")

# Check for duplicate Country-Year combinations (more important)
n_country_year_dupes = df.duplicated(subset=["Country", "Year"]).sum()
print(f"Duplicate Country-Year pairs: {n_country_year_dupes}")

if n_country_year_dupes > 0:
    dupe_pairs = df[df.duplicated(subset=["Country", "Year"], keep=False)].sort_values(["Country", "Year"])
    print(f"\nCountries with duplicate year entries:")
    print(dupe_pairs.groupby("Country")["Year"].apply(list).to_string())

=== Duplicate Analysis ===
Fully duplicate rows: 0
Duplicate Country-Year pairs: 42

Countries with duplicate year entries:
Country
Australia    [2000, 2000, 2005, 2005, 2010, 2010, 2015, 201...
Brazil       [2000, 2000, 2005, 2005, 2010, 2010, 2015, 201...
China        [2000, 2000, 2005, 2005, 2010, 2010, 2015, 201...
Germany      [2000, 2000, 2005, 2005, 2010, 2010, 2015, 201...
India        [2000, 2000, 2005, 2005, 2010, 2010, 2015, 201...
Nigeria      [2000, 2000, 2005, 2005, 2010, 2010, 2015, 201...
Russia       [2000, 2000, 2005, 2005, 2010, 2010, 2015, 201...


**Understanding duplicates:**

Duplicate rows can inflate statistics (making trends appear stronger than they are), create misleading visualizations, and violate assumptions in statistical tests.

If duplicates exist, we need to investigate whether they're true duplicates (identical entries to remove), valid repeated measurements (keep but document), or data entry errors (needs correction).

## Section 3 - Data cleaning

We apply cleaning steps to address any issues found in the quality checks: remove duplicates, handle missing values, validate data types and value ranges.

In [8]:
print(f"Before cleaning: {len(df)} rows")

# Step 1: Remove duplicate Country-Year pairs (keep first occurrence)
before_dupes = len(df)
df = df.drop_duplicates(subset=["Country", "Year"], keep="first").reset_index(drop=True)
print(f"Step 1 - Removed {before_dupes - len(df)} duplicate Country-Year rows → {len(df)} rows remain")

# Step 2: Drop rows with missing target variable (temperature)
before_na = len(df)
df = df.dropna(subset=["Avg_Temperature_degC"])
print(f"Step 2 - Removed {before_na - len(df)} rows with missing temperature → {len(df)} rows remain")

# Step 3: Fill remaining missing numeric values with country-level medians
numeric_cols = df.select_dtypes(include="number").columns.tolist()
for col in numeric_cols:
    if df[col].isna().sum() > 0:
        filled = df[col].isna().sum()
        df[col] = df.groupby("Country")[col].transform(lambda x: x.fillna(x.median()))
        # If still missing (country has no data), fill with global median
        df[col] = df[col].fillna(df[col].median())
        print(f"Step 3 - Filled {filled} missing values in {col} with country/global median")

# Step 4: Validate data types
df["Year"] = df["Year"].astype(int)
df["Population"] = df["Population"].astype(int)
print(f"\nStep 4 - Validated data types: Year(int), Population(int)")

# Step 5: Value range checks
print(f"\n=== Value Range Validation ===")
range_checks = {
    "Year": (1990, 2030),
    "Avg_Temperature_degC": (-20, 50),
    "CO2_Emissions_tons_per_capita": (0, 50),
    "Sea_Level_Rise_mm": (0, 200),
    "Rainfall_mm": (0, 5000),
    "Renewable_Energy_pct": (0, 100),
    "Forest_Area_pct": (0, 100),
    "Extreme_Weather_Events": (0, 200),
}
for col, (lo, hi) in range_checks.items():
    if col in df.columns:
        outliers = ((df[col] < lo) | (df[col] > hi)).sum()
        status = "✓" if outliers == 0 else f"⚠ {outliers} out-of-range"
        print(f"  {col}: {status} (expected {lo}–{hi})")

print(f"\nAfter cleaning: {len(df)} rows, {len(df.columns)} columns")

Before cleaning: 156 rows
Step 1 - Removed 42 duplicate Country-Year rows → 114 rows remain
Step 2 - Removed 0 rows with missing temperature → 114 rows remain

Step 4 - Validated data types: Year(int), Population(int)

=== Value Range Validation ===
  Year: ✓ (expected 1990–2030)
  Avg_Temperature_degC: ✓ (expected -20–50)
  CO2_Emissions_tons_per_capita: ✓ (expected 0–50)
  Sea_Level_Rise_mm: ✓ (expected 0–200)
  Rainfall_mm: ✓ (expected 0–5000)
  Renewable_Energy_pct: ✓ (expected 0–100)
  Forest_Area_pct: ✓ (expected 0–100)
  Extreme_Weather_Events: ✓ (expected 0–200)

After cleaning: 114 rows, 10 columns


**What the cleaning achieved:**

The steps above ensure the dataset is ready for analysis. Here is what each step does in plain English:

1. **Duplicate removal** — Some countries had more than one row for the same year (e.g. two entries for Brazil in 2010). We kept only the first occurrence so each Country-Year pair appears exactly once. This prevents inflated statistics and misleading trends.
2. **Missing temperature rows dropped** — Temperature is our main variable of interest (the "target"). Rows without it cannot contribute to any analysis, so they are removed entirely.
3. **Median imputation** — For other numeric columns (CO2, rainfall, etc.), any gaps are filled with the median value for that country. If a country has no data at all for a column, the global median is used as a fallback. Medians are preferred over means because they are less affected by extreme values.
4. **Type validation** — Year and Population are converted to whole numbers (integers). This avoids display issues like "2005.0" and ensures correct sorting.
5. **Range checks** — Each column is checked against sensible boundaries (e.g. temperatures between −20°C and 50°C, percentages between 0% and 100%). Any values outside these ranges are flagged so we can investigate further.

After cleaning, the dataset has no missing values, no duplicate Country-Year pairs, and all values fall within expected ranges. This is the single source of truth used by all downstream notebooks and the dashboard.

## Section 4 - Save cleaned output

We store a versioned cleaned dataset for downstream EDA, hypothesis tests, and modeling.

**Why versioning matters:** We save to `data/processed/v1/` so future updates go to v2, v3, etc., preserving full history and reproducibility.

In [9]:
clean_path = "data/processed/v1/environmental_trends_clean.csv"
df.to_csv(clean_path, index=False)

# Validate saved file
saved_df = pd.read_csv(clean_path)
assert len(saved_df) == len(df), f"Row count mismatch: saved {len(saved_df)}, expected {len(df)}"
assert list(saved_df.columns) == list(df.columns), "Column mismatch between saved and cleaned data"

print(f"✓ Saved {len(saved_df)} rows × {len(saved_df.columns)} columns to {clean_path}")
print(f"✓ Countries: {saved_df['Country'].nunique()}")
print(f"✓ Year range: {saved_df['Year'].min()} – {saved_df['Year'].max()}")
print(f"✓ Missing values: {saved_df.isna().sum().sum()}")
print(f"✓ Duplicate rows: {saved_df.duplicated().sum()}")

✓ Saved 114 rows × 10 columns to data/processed/v1/environmental_trends_clean.csv
✓ Countries: 19
✓ Year range: 2000 – 2024
✓ Missing values: 0
✓ Duplicate rows: 0


## Summary

**Cleaning steps applied:**
1. Removed duplicate Country-Year pairs (kept first occurrence)
2. Dropped rows with missing temperature (target variable)
3. Imputed remaining missing numerics with country-level medians (global median as fallback)
4. Validated data types (Year and Population as integers)
5. Checked all numeric columns against expected value ranges

The cleaned dataset is now saved to `data/processed/v1/environmental_trends_clean.csv` and serves as the single source of truth for all downstream analysis (EDA, hypothesis testing, modeling, and dashboard).